<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson1/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# EDA 
<!--- @wandbcode{course-lesson1} -->

In this notebook, we will download a sample of the [BDD100K](https://www.bdd100k.com/) semantic segmentation dataset and use W&B Artifacts and Tables to version and analyze our data. 

In [33]:
DEBUG = False # set this flag to True to use a small subset of data for testing

In [43]:
import sys
sys.path.insert(0,'/content/')

In [44]:
!pip install wandb
!pip install fastai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
pip uninstall params 

In [46]:
from fastai.vision.all import *
import params

import wandb

In [37]:
!wandb login

wandb: Currently logged in as: abhaylal (teamphoton). Use `wandb login --relogin` to force relogin


We have defined some global configuration parameters in the `params.py` file. `ENTITY` should correspond to your W&B Team name if you work in a team, replace it with `None` if you work individually. 

In the section below, we will use `untar_data` function from `fastai` to download and unzip our datasets. 

In [38]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [39]:
path = Path(untar_data(URL, force_download=True))

In [40]:
path.ls()

(#3) [Path('/root/.fastai/data/bdd_simple_1k/labels'),Path('/root/.fastai/data/bdd_simple_1k/images'),Path('/root/.fastai/data/bdd_simple_1k/LICENSE.txt')]

Here we define several functions to help us process the data and upload it as a `Table` to W&B. 

In [41]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

We will start a new W&B `run` and put everything into a raw Artifact.

In [50]:
run = wandb.init(project="Efficient Mlops", entity=None, job_type="upload")
raw_data_at = wandb.Artifact('bdd_simple_1k', type="raw_data")

In [51]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

ArtifactManifestEntry(path='LICENSE.txt', digest='X+6ZFkDOlnKesJCNt20yRg==', ref=None, birth_artifact_id=None, size=1594, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmp8ew09osv')

Let's add the images and label masks.

In [52]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/images)... Done. 0.8s
wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/labels)... Done. 0.6s


Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table. 

In [53]:
image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [54]:
table = _create_table(image_files, {i:c for i,c in enumerate(['background', 'road', 'traffic light', 'traffic sign', 'person', 'vehicle', 'bicycle'])} )

Finally, we will add the Table to our Artifact, log it to W&B and finish our `run`. 

In [55]:
raw_data_at.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='AyVoXFvnYqn1jlSxbEobyg==', ref=None, birth_artifact_id=None, size=588824, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmpjt54apmy')

In [56]:
run.log_artifact(raw_data_at)
run.finish()